# 继承的优缺点

本章分析了Python中的继承。主要是分析了在Python中进行继承应当注意的问题，并以Tkinter和Django为例讲解如何更好的应用多重继承。

## 子类化内置类型的麻烦

子类化内置类型绝不是一个好主意。其理由在于：Python中内置类型（具体来说，是使用C语言实现的内置类型）被其子类覆盖的方法的调用充满不确定性，这一不确定性指的是一些情况下会调用子类覆盖后的方法，另一些情况下则会调用内置类型原本的方法。

下述例子中，DoppelDict继承自dict，并且覆写了\_\_setitem\_\_方法。从示例中能够观察到上述的不确定性。

DoppelDict在初始化时，调用的是dict的\_\_setitem\_\_；在直接添加元素时使用的是覆写后的\_\_setitem\_\_；在update时则又是调用的dict的\_\_setitem\_\_。这种不确定性会在使用过程中造成相当程度上的困扰，并且难Debug。

MyDoppelDict则继承自UserDict。从示例中可以观察到，MyDoppelDict的行为完全正常，所有功能都符合预期。

In [6]:
from collections import UserDict

class DoppelDict(dict):

    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

class MyDoppelDict(UserDict):

    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [7]:
dd = DoppelDict(one=1)
print(dd)
dd["two"] = 2
print(dd)
dd.update(three=3)
print(dd)

Mydd = MyDoppelDict(one=1)
print(Mydd)
Mydd["two"] = 2
print(Mydd)
Mydd.update(three=3)
print(Mydd)

{'one': 1}
{'one': 1, 'two': [2, 2]}
{'one': 1, 'two': [2, 2], 'three': 3}
{'one': [1, 1]}
{'one': [1, 1], 'two': [2, 2]}
{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}


## 多重继承中的方法解析顺序

Python支持多重继承，因此也需要处理多重继承中可能出现的命名冲突。

对此，Python提供了类名限定方法来避免调用时的歧义。此外，Python中多重继承的方法解析顺序是可以预测的，并且也可以通过查看\_\_mro\_\_属性以得知特定类的方法解析顺序。

若想在方法中将调用委托给父类，可以使用内置的super()方法根据方法解析顺序调用父类方法，或者也可以直接指定调用某个父类的方法。

值得一提的是，Python中多重继承的方法解析顺序使用C3算法生成

In [11]:
class A:
    def ping(self):
        print("ping:", self)

class B(A):
    def pong(self):
        print("pong:", self)

class C(A):
    def pong(self):
        print("PONG:", self)

class D(B, C):

    """
    方法解析顺序：
    D.__mro__: 
    <class '__main__.D'>, 
    <class '__main__.B'>, 
    <class '__main__.C'>, 
    <class '__main__.A'>, 
    <class 'object'>
    """

    def ping(self):

        # 调用A的ping
        super().ping()
        print("post-ping:", self)
    
    def pingpong(self):

        # 调用A的ping()
        self.ping()
        # 调用A的ping()
        super().ping()
        # 调用B的pong()
        self.pong()
        # 调用B的pong()
        super().pong()
        # 调用C的pong()
        C.pong(self)

d = D()
print(D.__mro__)
d.pingpong()

(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)
ping: <__main__.D object at 0x0000022DC8853088>
post-ping: <__main__.D object at 0x0000022DC8853088>
ping: <__main__.D object at 0x0000022DC8853088>
pong: <__main__.D object at 0x0000022DC8853088>
pong: <__main__.D object at 0x0000022DC8853088>
PONG: <__main__.D object at 0x0000022DC8853088>


## 关于多重继承的一些建议

如何设计继承的层次关系仍没有完整的理论指导，但是本章提出了一些建议：

1. 区分接口继承和实现继承
    接口继承和实现继承的目的不同
    接口继承主要是设计了子类的接口（支柱）
    实现继承则是为了尽量避免代码重复（细节）
2. 使用抽象基类显式表示接口
    这一条建议那些仅定义接口的类应当被显式定义为抽象基类（个人持保留意见）
3. 通过混入重用代码
    混入类值得的是那些仅提供实现，而不体现接口的类。这些类很可能被大量毫不相关的子类继承。
    原则上混入类不定义新类型，仅是对方法进行打包以方便重用。
    因此混入类不应当被实例化，并且具体类不应当仅继承自混入类
4. 在名称中显式表示混入类
    Python并没有显式定义混入类的语法，因此作者建议在名称中加入Mixin后缀以表明该类是一个混入类
5. 不要子类化多个具体类
    具体类应当最多只有一个具体超类。More Effective C++的作者Scott Meyer则认为任何非尾端类都不应当是具体类
6. 聚合类
    聚合类：如果一个类的结构主要继承自混入，自身没有添加结构或者行为，那么这样的类称为聚合类。聚合类仅是一种组合
7. 优先使用对象组合，而不是类继承
    组合能使设计更灵活，子类化则是一种强耦合

## Tkinter中的多重继承

Tkinter的底层是Tcl的GUI工具包Tk（非面向对象），并且Tkinter从Python1.1版本开始就是标准库之一。由于年代久远，Tkinter中类的继承关系和上述给出的建议基本不符合。

Tkinter中封装GUI应用逻辑的Tk类均继承自Wm和Misc。从上述建议的角度来说，Misc类既不是抽象类，也不是混入类，并且该类有超过100个方法。Misc为每一个小组件提供各种功能。由于这种庞杂的设计，这一继承结构中会出现令人费解的“组合”，比如：为按钮提供计时器，为滚动条提供文字选择功能等。本书作者认为Misc类应当做进一步的层次划分和功能拆分。

作为用户，上述继承关系无论多么混乱，实际上没有什么影响（都是实现细节）。但是在查看或者检索需要的方法时则会非常头疼 —— 如何从下述Button的210个属性中选出当前项目可能用到的属性？（只能看文档了）

In [14]:
import tkinter as tk

print(len(dir(tk.Button)))
print(dir(tk.Button))

210
['_Misc__winfo_getint', '_Misc__winfo_parseitem', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_bind', '_configure', '_displayof', '_do', '_getboolean', '_getconfigure', '_getconfigure1', '_getdoubles', '_getints', '_grid_configure', '_gridconvvalue', '_last_child_ids', '_nametowidget', '_noarg_', '_options', '_register', '_report_exception', '_root', '_setup', '_subst_format', '_subst_format_str', '_substitute', '_tclCommands', '_windowingsystem', 'after', 'after_cancel', 'after_idle', 'anchor', 'bbox', 'bell', 'bind', 'bind_all', 'bind_class', 'bindtags', 'cget', 'clipboard_append', 'clipboard_clear', 'clipboard_get', 'columnconfigure', 'config', 'configur

## 总结

1. 为了避免行为上的不确定性，应当尽可能避免继承自内置类型，而应当选择继承自collections中的UserList、UserDict以及UserString
2. 命名冲突在多重继承中时有发生，Python使用特定的算法生成方法解析顺序，并且可以通过类的\_\_mro\_\_属性查看这一顺序。根据方法解析顺序，Python开发者能够精准预测调用方法的结果
3. 虽然仍没有完整的理论指导多重继承，但是本书作者给出了一些相当有益的建议，并且以Tkinter和Django为例分析如何更好的使用多重继承。总的来说，应当尽可能区分“接口”和“实现”，并且谨慎使用混入类。
